# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-d745711291-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series ("Association Football Club explorative search") 

Consider the following exploratory information need:

> investigate the association football clubs and find the main BGPs related to this sport. Compare clubs in countries and championships won. Explore also the participation of the countries in the FIFA World Cup.

**Important consideration: in wikidata, we are talking of ASSOCIATION FOOTBALL**

Q11571## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q11571`| Cristiano Ronaldo       | node |





Also consider

```
wd:Q11571 ?p ?obj .
```

is the BGP to retrieve all **properties of Cristiano Ronaldo**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for association football club

2. Identify the BGP for association football league

3. Return for each north american country the number of association football clubs

4. Identify the BGP for the UEFA Champions League.

5. Consider the countries which have at least two different clubs of their nationality who won the UEFA Champions League. Return the list of these countries with the number of different clubs who won the UEFA Champions League (the result set must be triples of country IRI, label and #clubs).

6. Return for each european country the top level league (the result set must be a list of country IRI and label, and league IRI and label).

7. Identify the BGP of the FIFA World Cup

8. Consider countries who participated at the FIFA World Cup more than 10 times. Return these countries with the number of apparitions (the result set must be triples of country IRI, label and #apparitions).

9. Consider the countries that won the FIFA World Cup at least twice. Find the association football clubs of those countries won the UEFA Champions League (the result set must be couples of clubs IRI and label).


## Task 1
Identify the BGP for association football club

In [2]:
# query example
#[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
#[('p', 'http://www.wikidata.org/entity/Q2736'), ('name', 'association football')]
#[('p', 'http://www.wikidata.org/entity/Q5611338'), ('name', 'Category:Association football')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q2736 ?p ?obj.
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3719'), ('name', 'regulated by')]
[('p', 'http://www.wikidata.org/prop/direct/P10'), ('name', 'video')]
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1190'), ('name', 'Universal Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1282'), ('name', 'OpenStreetMap tag or key')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('

In [3]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q11571 ?obj ?p.
   # get the label
   ?p sc:name ?name.
   FILTER(regex(?name,"association.*"))
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q2736'), ('name', 'association football')]
[('p', 'http://www.wikidata.org/entity/Q937857'), ('name', 'association football player')]
[('p', 'http://www.wikidata.org/entity/Q267245'), ('name', 'Portugal national association football team')]
3


In [4]:
# query example
# [('p', 'http://www.wikidata.org/entity/Q267245'), ('name', 'Portugal national association football team')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q267245 wdt:P31 ?p.
   # get the label
   ?p sc:name ?name.
   FILTER(regex(?name,"association.*"))
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q6979593'), ('name', 'national association football team')]
1


In [5]:
# query example
# [('p', 'http://www.wikidata.org/entity/Q267245'), ('name', 'Portugal national association football team')]
# [('p', 'http://www.wikidata.org/entity/Q6979593'), ('name', 'national association football team')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q6979593 wdt:P31 ?p.
   # get the label
   ?p sc:name ?name.
   FILTER(regex(?name,"association.*"))
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
Empty


Final query for this task

In [6]:
# write your final query
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q937857 ?p ?obj. #association football player (Q937857)
   # get the label
   ?obj sc:name ?name.
   FILTER(regex(?name,"association football club"))
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q476028'), ('name', 'association football club')]
1


## Task 2
Identify the BGP for association football league

In [2]:
# write your queries
# association football league (Q15991303)
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q15991303 ?p ?obj . 
   # get the label
   ?p sc:name ?name.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('name', 'KBpedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
9


In [3]:
# write your queries
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q11571 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('name', 'medical condition')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http:

In [4]:
# write your queries
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q11571 wdt:P118 ?obj .
   # get the label
   ?obj sc:name ?name.
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q182994'), ('name', 'Liga Portugal')]
[('obj', 'http://www.wikidata.org/entity/Q15804'), ('name', 'Serie A')]
[('obj', 'http://www.wikidata.org/entity/Q324867'), ('name', 'La Liga')]
[('obj', 'http://www.wikidata.org/entity/Q9448'), ('name', 'Premier League')]
4


In [5]:
# write your queries
# [('obj', 'http://www.wikidata.org/entity/Q15804'), ('name', 'Serie A')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q15804 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1350'), ('name', 'number of matches played/races/starts')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2003'), ('name', 'Instagram username')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('name', 'Facebook ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class')]
[('p',

In [73]:
# write your queries
# [('obj', 'http://www.wikidata.org/entity/Q15804'), ('name', 'Serie A')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q15804 ?p ?obj .
   # get the label
   ?obj sc:name ?name. # we are looking for objects of wd:15804
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q194052'), ('name', 'Serie A')]
[('obj', 'http://www.wikidata.org/entity/Q6807104'), ('name', 'Category:Serie A')]
[('obj', 'http://www.wikidata.org/entity/Q122'), ('name', 'August')]
[('obj', 'http://www.wikidata.org/entity/Q2736'), ('name', 'association football')]
[('obj', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
[('obj', 'http://www.wikidata.org/entity/Q631'), ('name', 'FC Internazionale Milano')]
[('obj', 'http://www.wikidata.org/entity/Q1478437'), ('name', 'association football competition')]
[('obj', 'http://www.wikidata.org/entity/Q15991303'), ('name', 'association football league')]
[('obj', 'http://www.wikidata.org/entity/Q31930761'), ('name', "men's association football")]
[('obj', 'http://www.wikidata.org/entity/Q3270632'), ('name', 'national championship')]
[('obj', 'http://www.wikidata.org/entity/Q953425'), ('name', 'Italian association football league system')]
[('obj', 'http://www.wikidata.org/entity/Q242

In [73]:
# write your queries
# [('obj', 'http://www.wikidata.org/entity/Q15804'), ('name', 'Serie A')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q15804 ?p ?obj .
   # get the label
   ?obj sc:name ?name. # we are looking for objects of wd:15804
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q194052'), ('name', 'Serie A')]
[('obj', 'http://www.wikidata.org/entity/Q6807104'), ('name', 'Category:Serie A')]
[('obj', 'http://www.wikidata.org/entity/Q122'), ('name', 'August')]
[('obj', 'http://www.wikidata.org/entity/Q2736'), ('name', 'association football')]
[('obj', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
[('obj', 'http://www.wikidata.org/entity/Q631'), ('name', 'FC Internazionale Milano')]
[('obj', 'http://www.wikidata.org/entity/Q1478437'), ('name', 'association football competition')]
[('obj', 'http://www.wikidata.org/entity/Q15991303'), ('name', 'association football league')]
[('obj', 'http://www.wikidata.org/entity/Q31930761'), ('name', "men's association football")]
[('obj', 'http://www.wikidata.org/entity/Q3270632'), ('name', 'national championship')]
[('obj', 'http://www.wikidata.org/entity/Q953425'), ('name', 'Italian association football league system')]
[('obj', 'http://www.wikidata.org/entity/Q242

In [76]:
# write your queries
# [('obj', 'http://www.wikidata.org/entity/Q15804'), ('name', 'Serie A')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q15804 ?p ?obj .
   # get the label
   ?obj sc:name ?name. # we are looking for objects of wd:15804
   filter(regex(?name,"association football league"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q15991303'), ('name', 'association football league')]
[('obj', 'http://www.wikidata.org/entity/Q953425'), ('name', 'Italian association football league system')]
2


Final query for this task

In [76]:
# write your final query
# [('obj', 'http://www.wikidata.org/entity/Q15804'), ('name', 'Serie A')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q15804 ?p ?obj .
   # get the label
   ?obj sc:name ?name. # we are looking for objects of wd:15804
   filter(regex(?name,"association football league"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q15991303'), ('name', 'association football league')]
[('obj', 'http://www.wikidata.org/entity/Q953425'), ('name', 'Italian association football league system')]
2


## Task 3
Return for each north american country the number of association football clubs

In [78]:
# write your queries
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q476028 ?p ?obj .
   # get the label
   ?obj sc:name ?name. # we are looking for objects of wd:15804
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q51481377'), ('name', "women's association football club")]
[('obj', 'http://www.wikidata.org/entity/Q15944511'), ('name', 'association football team')]
[('obj', 'http://www.wikidata.org/entity/Q2736'), ('name', 'association football')]
[('obj', 'http://www.wikidata.org/entity/Q628099'), ('name', 'association football manager')]
[('obj', 'http://www.wikidata.org/entity/Q937857'), ('name', 'association football player')]
[('obj', 'http://www.wikidata.org/entity/Q714581'), ('name', 'Nea Salamis Famagusta FC')]
[('obj', 'http://www.wikidata.org/entity/Q167270'), ('name', 'trademark')]
[('obj', 'http://www.wikidata.org/entity/Q17270000'), ('name', 'football club')]
[('obj', 'http://www.wikidata.org/entity/Q28083137'), ('name', 'football team')]
[('obj', 'http://www.wikidata.org/entity/P5424'), ('name', 'Lega Serie A team ID')]
[('obj', 'http://www.wikidata.org/entity/Q5614332'), ('name', 'Category:Association football clubs')]
[('obj', 'http

In [79]:
# [('obj', 'http://www.wikidata.org/entity/Q5614332'), ('name', 'Category:Association football clubs')]
# write your queries
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q5614332 ?p ?obj .
   # get the label
   ?obj sc:name ?name. # we are looking for objects of wd:15804
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q4167836'), ('name', 'Wikimedia category')]
[('obj', 'http://www.wikidata.org/entity/Q476028'), ('name', 'association football club')]
[('obj', 'http://www.wikidata.org/entity/Q6384029'), ('name', 'Category:Association football teams')]
3


In [80]:
# [('obj', 'http://www.wikidata.org/entity/Q5614332'), ('name', 'Category:Association football clubs')]
# write your queries
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q5614332 ?p ?obj .
   # get the label
   ?p sc:name ?name. # we are looking for objects of wd:15804
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
3


In [81]:
# [('obj', 'http://www.wikidata.org/entity/Q5614332'), ('name', 'Category:Association football clubs')]
# write your queries
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q5614332 ?p ?obj .
   # get the label
   ?obj sc:name ?name. # we are looking for objects of wd:15804
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q4167836'), ('name', 'Wikimedia category')]
[('obj', 'http://www.wikidata.org/entity/Q476028'), ('name', 'association football club')]
[('obj', 'http://www.wikidata.org/entity/Q6384029'), ('name', 'Category:Association football teams')]
3


In [83]:
# [('obj', 'http://www.wikidata.org/entity/Q476028'), ('name', 'association football club')]
# write your queries
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q476028 ?p ?obj .
   # get the label
   ?obj sc:name ?name. # we are looking for objects of wd:15804
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q51481377'), ('name', "women's association football club")]
[('obj', 'http://www.wikidata.org/entity/Q15944511'), ('name', 'association football team')]
[('obj', 'http://www.wikidata.org/entity/Q2736'), ('name', 'association football')]
[('obj', 'http://www.wikidata.org/entity/Q628099'), ('name', 'association football manager')]
[('obj', 'http://www.wikidata.org/entity/Q937857'), ('name', 'association football player')]
[('obj', 'http://www.wikidata.org/entity/Q714581'), ('name', 'Nea Salamis Famagusta FC')]
[('obj', 'http://www.wikidata.org/entity/Q167270'), ('name', 'trademark')]
[('obj', 'http://www.wikidata.org/entity/Q17270000'), ('name', 'football club')]
[('obj', 'http://www.wikidata.org/entity/Q28083137'), ('name', 'football team')]
[('obj', 'http://www.wikidata.org/entity/P5424'), ('name', 'Lega Serie A team ID')]
[('obj', 'http://www.wikidata.org/entity/Q5614332'), ('name', 'Category:Association football clubs')]
[('obj', 'http

In [85]:
# [('obj', 'http://www.wikidata.org/entity/Q476028'), ('name', 'association football club')]
# write your queries
# let's discover all association football club
queryString = """
SELECT DISTINCT ?subject ?sName ?p ?pName
WHERE {
   # bind something
   ?subject ?p wd:Q476028 .
   
   # get the label
   ?subject sc:name ?sName.
   ?p sc:name ?pName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('subject', 'http://www.wikidata.org/entity/Q5611964'), ('sName', 'Template:Infobox football club'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic')]
[('subject', 'http://www.wikidata.org/entity/Q51481377'), ('sName', "women's association football club"), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('subject', 'http://www.wikidata.org/entity/Q15944511'), ('sName', 'association football team'), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('subject', 'http://www.wikidata.org/entity/Q28083137'), ('sName', 'football team'), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('subject', 'http://www.wikidata.org/entity/Q103231176'), ('sName', "men's association football club"), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('subject', 'http://www.wikidata.org/entity/Q51481377'), ('sName', "women's associati

In [85]:
# [('obj', 'http://www.wikidata.org/entity/Q476028'), ('name', 'association football club')]
# write your queries
# let's discover all association football clubs
queryString = """
SELECT DISTINCT ?subject ?sName ?p ?pName
WHERE {
   # bind something
   ?subject ?p wd:Q476028 .
   
   # get the label
   ?subject sc:name ?sName.
   ?p sc:name ?pName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('subject', 'http://www.wikidata.org/entity/Q5611964'), ('sName', 'Template:Infobox football club'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic')]
[('subject', 'http://www.wikidata.org/entity/Q51481377'), ('sName', "women's association football club"), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('subject', 'http://www.wikidata.org/entity/Q15944511'), ('sName', 'association football team'), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('subject', 'http://www.wikidata.org/entity/Q28083137'), ('sName', 'football team'), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('subject', 'http://www.wikidata.org/entity/Q103231176'), ('sName', "men's association football club"), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('subject', 'http://www.wikidata.org/entity/Q51481377'), ('sName', "women's associati

In [86]:
# write your queries
# let's find continents P30
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   ?s wdt:P30 ?obj .
   
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q60140'), ('name', 'Indian subcontinent')]
[('obj', 'http://www.wikidata.org/entity/Q55643'), ('name', 'Oceania')]
[('obj', 'http://www.wikidata.org/entity/Q19464773'), ('name', 'Northern America and Mexico')]
[('obj', 'http://www.wikidata.org/entity/Q1239'), ('name', 'Indian Ocean')]
[('obj', 'http://www.wikidata.org/entity/Q15'), ('name', 'Africa')]
[('obj', 'http://www.wikidata.org/entity/Q18'), ('name', 'South America')]
[('obj', 'http://www.wikidata.org/entity/Q4412'), ('name', 'West Africa')]
[('obj', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
[('obj', 'http://www.wikidata.org/entity/Q48'), ('name', 'Asia')]
[('obj', 'http://www.wikidata.org/entity/Q49'), ('name', 'North America')]
[('obj', 'http://www.wikidata.org/entity/Q51'), ('name', 'Antarctica')]
[('obj', 'http://www.wikidata.org/entity/Q538'), ('name', 'Insular Oceania')]
[('obj', 'http://www.wikidata.org/entity/Q5401'), ('name', 'Eurasia')]
[('obj', 'http://

In [88]:
# write your queries
# let's find continents P30
# [('obj', 'http://www.wikidata.org/entity/Q49'), ('name', 'North America')]
queryString = """
SELECT DISTINCT ?club ?name
WHERE {
   # bind something
   ?club wdt:P31 wd:Q476028; #instance of P31
       wdt:P17 ?country.
       
    ?country wdt:P30 wd:Q49 . # only those countries 'P30' which are in North America 'Q49'
   # get the label
   ?club sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('club', 'http://www.wikidata.org/entity/Q7280157'), ('name', "Radford Highlanders men's soccer")]
[('club', 'http://www.wikidata.org/entity/Q15876988'), ('name', 'Grenades FC')]
[('club', 'http://www.wikidata.org/entity/Q3930989'), ('name', 'Real Colorado Foxes')]
[('club', 'http://www.wikidata.org/entity/Q4816106'), ('name', 'Atlanta Magic')]
[('club', 'http://www.wikidata.org/entity/Q4852791'), ('name', 'Baltimore Bohemians')]
[('club', 'http://www.wikidata.org/entity/Q5020506'), ('name', 'California Gold')]
[('club', 'http://www.wikidata.org/entity/Q5087877'), ('name', 'Chattanooga Express')]
[('club', 'http://www.wikidata.org/entity/Q5095562'), ('name', 'Chicago Croatian')]
[('club', 'http://www.wikidata.org/entity/Q5266043'), ('name', 'Waza Flo')]
[('club', 'http://www.wikidata.org/entity/Q6116983'), ('name', 'Jackson Chargers')]
[('club', 'http://www.wikidata.org/entity/Q7013918'), ('name', 'New York Red Bulls U-23')]
[('club', 'http://www.wikidata.org/entity/Q7389397')

In [89]:
# write your queries
# let's count the clubs
# let's find continents P30
# [('obj', 'http://www.wikidata.org/entity/Q49'), ('name', 'North America')]
queryString = """
SELECT DISTINCT ?country (COUNT(?club) AS ?club)
WHERE {
   # bind something
   ?club wdt:P31 wd:Q476028; #instance of P31
       wdt:P17 ?country.
       
    ?country wdt:P30 wd:Q49 . # only those countries 'P30' which are in North America 'Q49'
   # get the label
   ?club sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q757'), ('club', '11')]
[('country', 'http://www.wikidata.org/entity/Q804'), ('club', '32')]
[('country', 'http://www.wikidata.org/entity/Q778'), ('club', '9')]
[('country', 'http://www.wikidata.org/entity/Q786'), ('club', '13')]
[('country', 'http://www.wikidata.org/entity/Q754'), ('club', '22')]
[('country', 'http://www.wikidata.org/entity/Q29999'), ('club', '3')]
[('country', 'http://www.wikidata.org/entity/Q769'), ('club', '1')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('club', '1193')]
[('country', 'http://www.wikidata.org/entity/Q783'), ('club', '90')]
[('country', 'http://www.wikidata.org/entity/Q242'), ('club', '35')]
[('country', 'http://www.wikidata.org/entity/Q800'), ('club', '45')]
[('country', 'http://www.wikidata.org/entity/Q1183'), ('club', '1')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('club', '136')]
[('country', 'http://www.wikidata.org/entity/Q223'), ('club', '12')]
[('country', 'http://www.w

In [96]:
# write your queries
# let's count the clubs
# let's find continents P30
# let's find with country names
# [('obj', 'http://www.wikidata.org/entity/Q49'), ('name', 'North America')]
queryString = """
SELECT DISTINCT ?country (COUNT(?club) AS ?club)
WHERE {
   # bind something
   ?club wdt:P31 wd:Q476028; #instance of P31
       wdt:P17 ?country.
       
    ?country wdt:P30 wd:Q49 . # only those countries 'P30' which are in North America 'Q49'
   # get the label
   ?country sc:name ?name.
   #?club sc:name ?name.
}
GROUP BY (?country)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q757'), ('club', '11')]
[('country', 'http://www.wikidata.org/entity/Q804'), ('club', '36')]
[('country', 'http://www.wikidata.org/entity/Q778'), ('club', '45')]
[('country', 'http://www.wikidata.org/entity/Q786'), ('club', '13')]
[('country', 'http://www.wikidata.org/entity/Q754'), ('club', '22')]
[('country', 'http://www.wikidata.org/entity/Q29999'), ('club', '3')]
[('country', 'http://www.wikidata.org/entity/Q769'), ('club', '1')]
[('country', 'http://www.wikidata.org/entity/Q242'), ('club', '100')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('club', '1199')]
[('country', 'http://www.wikidata.org/entity/Q783'), ('club', '93')]
[('country', 'http://www.wikidata.org/entity/Q800'), ('club', '91')]
[('country', 'http://www.wikidata.org/entity/Q1183'), ('club', '1')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('club', '138')]
[('country', 'http://www.wikidata.org/entity/Q784'), ('club', '54')]
[('country', 'http://www

In [98]:
# write your queries
# let's count the clubs
# let's find continents P30
# let's find with country names
# [('obj', 'http://www.wikidata.org/entity/Q49'), ('name', 'North America')]
# let's try group by country name
queryString = """
SELECT DISTINCT ?name (COUNT(?club) AS ?club)
WHERE {
   # bind something
   ?club wdt:P31 wd:Q476028; #instance of P31
       wdt:P17 ?country.
       
    ?country wdt:P30 wd:Q49 . # only those countries 'P30' which are in North America 'Q49'
   # get the label
   ?country sc:name ?name.
   #?club sc:name ?name.
}
GROUP BY (?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'Jamaica'), ('club', '30')]
[('name', 'Cuba'), ('club', '21')]
[('name', 'Mexico'), ('club', '355')]
[('name', 'Greenland'), ('club', '12')]
[('name', 'Panama'), ('club', '36')]
[('name', 'Barbados'), ('club', '10')]
[('name', 'Russian Empire'), ('club', '3')]
[('name', 'Saint Vincent and the Grenadines'), ('club', '11')]
[('name', 'Guatemala'), ('club', '45')]
[('name', 'Costa Rica'), ('club', '91')]
[('name', 'Dominican Republic'), ('club', '13')]
[('name', 'El Salvador'), ('club', '104')]
[('name', 'Saint Kitts and Nevis'), ('club', '10')]
[('name', 'United States of America'), ('club', '1199')]
[('name', 'Denmark'), ('club', '267')]
[('name', 'Haiti'), ('club', '66')]
[('name', 'Turks and Caicos Islands'), ('club', '12')]
[('name', 'Canada'), ('club', '138')]
[('name', 'Belize'), ('club', '100')]
[('name', 'The Bahamas'), ('club', '45')]
[('name', 'Honduras'), ('club', '93')]
[('name', 'Dominica'), ('club', '54')]
[('name', 'Trinidad and Tobago'), ('club', '22')]


Final query for this task

In [99]:
# write your final query
# write your queries
# let's count the clubs
# let's find continents P30
# let's find with country names
# [('obj', 'http://www.wikidata.org/entity/Q49'), ('name', 'North America')]
# let's try group by country name
queryString = """
SELECT DISTINCT ?name (COUNT(?club) AS ?club)
WHERE {
   # bind something
   ?club wdt:P31 wd:Q476028; #instance of P31
       wdt:P17 ?country.
       
    ?country wdt:P30 wd:Q49 . # only those countries 'P30' which are in North America 'Q49'
   # get the label
   ?country sc:name ?name.
   #?club sc:name ?name.
}
GROUP BY (?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'Jamaica'), ('club', '30')]
[('name', 'Cuba'), ('club', '21')]
[('name', 'Mexico'), ('club', '355')]
[('name', 'Greenland'), ('club', '12')]
[('name', 'Panama'), ('club', '36')]
[('name', 'Barbados'), ('club', '10')]
[('name', 'Russian Empire'), ('club', '3')]
[('name', 'Saint Vincent and the Grenadines'), ('club', '11')]
[('name', 'Guatemala'), ('club', '45')]
[('name', 'Costa Rica'), ('club', '91')]
[('name', 'Dominican Republic'), ('club', '13')]
[('name', 'El Salvador'), ('club', '104')]
[('name', 'Saint Kitts and Nevis'), ('club', '10')]
[('name', 'United States of America'), ('club', '1199')]
[('name', 'Denmark'), ('club', '267')]
[('name', 'Haiti'), ('club', '66')]
[('name', 'Turks and Caicos Islands'), ('club', '12')]
[('name', 'Canada'), ('club', '138')]
[('name', 'Belize'), ('club', '100')]
[('name', 'The Bahamas'), ('club', '45')]
[('name', 'Honduras'), ('club', '93')]
[('name', 'Dominica'), ('club', '54')]
[('name', 'Trinidad and Tobago'), ('club', '22')]


## Task 4
Identify the BGP for the UEFA Champions League.

In [100]:
# write your queries
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q15804 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1350'), ('name', 'number of matches played/races/starts')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2003'), ('name', 'Instagram username')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('name', 'Facebook ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class')]
[('p',

In [101]:
# write your queries
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q15991303 ?p ?obj . #association football league
   # get the label
   ?p sc:name ?name.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('name', 'KBpedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
9


In [102]:
# write your queries
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q15991303 ?p ?obj . #association football league
   # get the label
   ?obj sc:name ?name.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/P2499'), ('name', 'league level above')]
[('obj', 'http://www.wikidata.org/entity/P2500'), ('name', 'league level below')]
[('obj', 'http://www.wikidata.org/entity/P3983'), ('name', 'sports league level')]
[('obj', 'http://www.wikidata.org/entity/Q26887310'), ('name', 'association football team season')]
[('obj', 'http://www.wikidata.org/entity/P17'), ('name', 'country')]
[('obj', 'http://www.wikidata.org/entity/Q2736'), ('name', 'association football')]
[('obj', 'http://www.wikidata.org/entity/Q1478437'), ('name', 'association football competition')]
[('obj', 'http://www.wikidata.org/entity/P641'), ('name', 'sport')]
[('obj', 'http://www.wikidata.org/entity/Q623109'), ('name', 'sports league')]
[('obj', 'http://www.wikidata.org/entity/P664'), ('name', 'organizer')]
[('obj', 'http://www.wikidata.org/entity/P1132'), ('name', 'number of participants')]
[('obj', 'http://www.wikidata.org/entity/Q6383739'), ('name', 'Category:Association foot

In [106]:
# write your queries
# try to filter on the champions league
queryString = """
SELECT DISTINCT ?object ?name
WHERE {
   # bind something
   ?subject wdt:P3450 ?object . #sports season of league or competition P3450
   # get the label
   ?object sc:name ?name.
   FILTER REGEX (?name, ".*Champions League.*")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q652498'), ('name', "EHF Women's Champions League")]
[('object', 'http://www.wikidata.org/entity/Q648474'), ('name', 'OFC Champions League')]
[('object', 'http://www.wikidata.org/entity/Q300098'), ('name', 'CEV Champions League')]
[('object', 'http://www.wikidata.org/entity/Q13416183'), ('name', '2015–16 UEFA Champions League')]
[('object', 'http://www.wikidata.org/entity/Q219261'), ('name', 'CAF Champions League')]
[('object', 'http://www.wikidata.org/entity/Q3240479'), ('name', 'African Handball Champions League')]
[('object', 'http://www.wikidata.org/entity/Q18740961'), ('name', '2016–17 UEFA Champions League')]
[('object', 'http://www.wikidata.org/entity/Q20972117'), ('name', '2017–18 UEFA Champions League')]
[('object', 'http://www.wikidata.org/entity/Q941782'), ('name', 'LEN Champions League')]
[('object', 'http://www.wikidata.org/entity/Q22340390'), ('name', 'Basketball Champions League')]
[('object', 'http://www.wikidata.org/e

In [108]:
# write your queries
# try to filter on the champions league
# [('object', 'http://www.wikidata.org/entity/Q18756'), ('name', 'UEFA Champions League')]
queryString = """
SELECT DISTINCT ?object ?name
WHERE {
   # bind something
   ?subject wdt:P3450 ?object . #sports season of league or competition P3450
   # get the label
   ?object sc:name ?name.
   FILTER(regex(?name, "UEFA Champions League.*"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q13416183'), ('name', '2015–16 UEFA Champions League')]
[('object', 'http://www.wikidata.org/entity/Q18740961'), ('name', '2016–17 UEFA Champions League')]
[('object', 'http://www.wikidata.org/entity/Q20972117'), ('name', '2017–18 UEFA Champions League')]
[('object', 'http://www.wikidata.org/entity/Q18756'), ('name', 'UEFA Champions League')]
4


Final query for this task

In [108]:
# write your final query
# try to filter on the champions league
# [('object', 'http://www.wikidata.org/entity/Q18756'), ('name', 'UEFA Champions League')]
queryString = """
SELECT DISTINCT ?object ?name
WHERE {
   # bind something
   ?subject wdt:P3450 ?object . #sports season of league or competition P3450
   # get the label
   ?object sc:name ?name.
   FILTER(regex(?name, "UEFA Champions League.*"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q13416183'), ('name', '2015–16 UEFA Champions League')]
[('object', 'http://www.wikidata.org/entity/Q18740961'), ('name', '2016–17 UEFA Champions League')]
[('object', 'http://www.wikidata.org/entity/Q20972117'), ('name', '2017–18 UEFA Champions League')]
[('object', 'http://www.wikidata.org/entity/Q18756'), ('name', 'UEFA Champions League')]
4


## Task 5
Consider the countries which have at least two different clubs of their nationality who won the UEFA Champions League. Return the list of these countries with the number of different clubs who won the UEFA Champions League (the result set must be triples of country IRI, label and #clubs).

In [110]:
# write your queries
# Let's find properties of Champions League Q18756
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q18756 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2003'), ('name', 'Instagram username')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('name', 'Facebook ID')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2257'), ('name', 'event interval')]
[('p', 'http:/

In [111]:
# write your queries
# Let's find properties of Champions League Q18756
# [('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q18756 wdt:P1346 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q631'), ('name', 'FC Internazionale Milano')]
1


In [112]:
# write your queries
# UEFA Champions League (Q20972117)
# [('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q20972117 wdt:P1346 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q8682'), ('name', 'Real Madrid CF')]
1


In [114]:
# write your queries
# UEFA Champions League (Q20972117)
# sports season of league or competition (P3450)
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q20972117 wdt:P3450 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q18756'), ('name', 'UEFA Champions League')]
1


In [115]:
# write your queries
# UEFA Champions League (Q20972117)
# sports season of league or competition (P3450)
# [('obj', 'http://www.wikidata.org/entity/Q18756'), ('name', 'UEFA Champions League')]
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   wd:Q18756 wdt:P3450 ?s .
   # get the label
   ?s sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [116]:
# write your queries
# UEFA Champions League (Q20972117)
# sports season of league or competition (P3450)
# [('obj', 'http://www.wikidata.org/entity/Q18756'), ('name', 'UEFA Champions League')]
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s wd:Q18756 wdt:P3450 .
   # get the label
   ?s sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [117]:
# write your queries
# UEFA Champions League (Q20972117)
# sports season of league or competition (P3450)
# [('obj', 'http://www.wikidata.org/entity/Q18756'), ('name', 'UEFA Champions League')]
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s wdt:P3450 wd:Q18756 .
   # get the label
   ?s sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q497052'), ('name', '1964–65 European Cup')]
[('s', 'http://www.wikidata.org/entity/Q500084'), ('name', '1963–64 European Cup')]
[('s', 'http://www.wikidata.org/entity/Q30608'), ('name', '2013–14 UEFA Champions League')]
[('s', 'http://www.wikidata.org/entity/Q13416183'), ('name', '2015–16 UEFA Champions League')]
[('s', 'http://www.wikidata.org/entity/Q269535'), ('name', '1991–92 European Cup')]
[('s', 'http://www.wikidata.org/entity/Q308996'), ('name', '2004–05 UEFA Champions League')]
[('s', 'http://www.wikidata.org/entity/Q696237'), ('name', '1988–89 European Cup')]
[('s', 'http://www.wikidata.org/entity/Q696245'), ('name', '1986–87 European Cup')]
[('s', 'http://www.wikidata.org/entity/Q219384'), ('name', '1955–56 European Cup')]
[('s', 'http://www.wikidata.org/entity/Q456097'), ('name', '2000–01 UEFA Champions League')]
[('s', 'http://www.wikidata.org/entity/Q18740961'), ('name', '2016–17 UEFA Champions League')]
[('s', 'http://www.w

In [118]:
# write your queries
# UEFA Champions League (Q20972117)
# sports season of league or competition (P3450)
# [('obj', 'http://www.wikidata.org/entity/Q18756'), ('name', 'UEFA Champions League')]
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s wdt:P3450 wd:Q18756 .
   # get the label
   ?s sc:name ?name.
   FILTER (regex(?name,"UEFA Champions League"))
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q30608'), ('name', '2013–14 UEFA Champions League')]
[('s', 'http://www.wikidata.org/entity/Q13416183'), ('name', '2015–16 UEFA Champions League')]
[('s', 'http://www.wikidata.org/entity/Q308996'), ('name', '2004–05 UEFA Champions League')]
[('s', 'http://www.wikidata.org/entity/Q456097'), ('name', '2000–01 UEFA Champions League')]
[('s', 'http://www.wikidata.org/entity/Q18740961'), ('name', '2016–17 UEFA Champions League')]
[('s', 'http://www.wikidata.org/entity/Q20972117'), ('name', '2017–18 UEFA Champions League')]
[('s', 'http://www.wikidata.org/entity/Q30032467'), ('name', '2018–19 UEFA Champions League')]
[('s', 'http://www.wikidata.org/entity/Q219705'), ('name', '2005–06 UEFA Champions League')]
[('s', 'http://www.wikidata.org/entity/Q184182'), ('name', '2009–10 UEFA Champions League')]
[('s', 'http://www.wikidata.org/entity/Q186489'), ('name', '2007–08 UEFA Champions League')]
[('s', 'http://www.wikidata.org/entity/Q189180'), ('nam

In [119]:
# write your queries
# UEFA Champions League (Q20972117)
# sports season of league or competition (P3450)
# [('obj', 'http://www.wikidata.org/entity/Q18756'), ('name', 'UEFA Champions League')]
# let's search for the UEFA winner (P1346)
queryString = """
SELECT DISTINCT ?s ?name ?winner ?winnerName
WHERE {
   # bind something
   ?s wdt:P3450 wd:Q18756;
       wdt:P1346 ?winner .
   # get the label
   ?s sc:name ?name.
   ?winner sc:name ?winnerName
  # FILTER (regex(?name,"UEFA Champions League"))
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q330674'), ('name', '1985–86 European Cup'), ('winner', 'http://www.wikidata.org/entity/Q179658'), ('winnerName', 'FCSB')]
[('s', 'http://www.wikidata.org/entity/Q308996'), ('name', '2004–05 UEFA Champions League'), ('winner', 'http://www.wikidata.org/entity/Q1130849'), ('winnerName', 'Liverpool F.C.')]
[('s', 'http://www.wikidata.org/entity/Q30032467'), ('name', '2018–19 UEFA Champions League'), ('winner', 'http://www.wikidata.org/entity/Q1130849'), ('winnerName', 'Liverpool F.C.')]
[('s', 'http://www.wikidata.org/entity/Q641465'), ('name', '1980–81 European Cup'), ('winner', 'http://www.wikidata.org/entity/Q1130849'), ('winnerName', 'Liverpool F.C.')]
[('s', 'http://www.wikidata.org/entity/Q196681'), ('name', '1977–78 European Cup'), ('winner', 'http://www.wikidata.org/entity/Q1130849'), ('winnerName', 'Liverpool F.C.')]
[('s', 'http://www.wikidata.org/entity/Q497020'), ('name', '1976–77 European Cup'), ('winner', 'http://www.wikidata.or

In [120]:
# write your queries
# UEFA Champions League (Q20972117)
# sports season of league or competition (P3450)
# [('obj', 'http://www.wikidata.org/entity/Q18756'), ('name', 'UEFA Champions League')]
# let's search for the UEFA winner (P1346)
queryString = """
SELECT DISTINCT ?s ?name ?winner ?winnerName
WHERE {
   # bind something
   ?s wdt:P3450 wd:Q18756;
       wdt:P1346 ?winner .
   # get the label
   ?s sc:name ?name.
   ?winner sc:name ?winnerName
   FILTER (regex(?name,"UEFA Champions League"))
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q308996'), ('name', '2004–05 UEFA Champions League'), ('winner', 'http://www.wikidata.org/entity/Q1130849'), ('winnerName', 'Liverpool F.C.')]
[('s', 'http://www.wikidata.org/entity/Q30032467'), ('name', '2018–19 UEFA Champions League'), ('winner', 'http://www.wikidata.org/entity/Q1130849'), ('winnerName', 'Liverpool F.C.')]
[('s', 'http://www.wikidata.org/entity/Q275673'), ('name', '1992–93 UEFA Champions League'), ('winner', 'http://www.wikidata.org/entity/Q132885'), ('winnerName', 'Olympique de Marseille')]
[('s', 'http://www.wikidata.org/entity/Q26170'), ('name', '1995–96 UEFA Champions League'), ('winner', 'http://www.wikidata.org/entity/Q1422'), ('winnerName', 'Juventus F.C.')]
[('s', 'http://www.wikidata.org/entity/Q455978'), ('name', '2002–03 UEFA Champions League'), ('winner', 'http://www.wikidata.org/entity/Q1543'), ('winnerName', 'A.C. Milan')]
[('s', 'http://www.wikidata.org/entity/Q4405'), ('name', '1993–94 UEFA Champions Leag

In [128]:
# write your queries
# UEFA Champions League (Q20972117)
# sports season of league or competition (P3450)
# [('obj', 'http://www.wikidata.org/entity/Q18756'), ('name', 'UEFA Champions League')]
# let's search for the UEFA winner (P1346)
# try to find country name
queryString = """
SELECT DISTINCT ?edition ?countryName ?winnerName
WHERE {
   # bind something
   ?s wdt:P3450 wd:Q18756;
       wdt:P1346 ?winner .
   ?winner wdt:P17 ?country .
   # get the label
   ?s sc:name ?edition .
   ?country sc:name ?countryName.
   ?winner sc:name ?winnerName
   # FILTER (regex(?name,"UEFA Champions League"))
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('edition', '1992–93 UEFA Champions League'), ('countryName', 'France'), ('winnerName', 'Olympique de Marseille')]
[('edition', '2004–05 UEFA Champions League'), ('countryName', 'United Kingdom'), ('winnerName', 'Liverpool F.C.')]
[('edition', '2018–19 UEFA Champions League'), ('countryName', 'United Kingdom'), ('winnerName', 'Liverpool F.C.')]
[('edition', '1980–81 European Cup'), ('countryName', 'United Kingdom'), ('winnerName', 'Liverpool F.C.')]
[('edition', '1977–78 European Cup'), ('countryName', 'United Kingdom'), ('winnerName', 'Liverpool F.C.')]
[('edition', '1976–77 European Cup'), ('countryName', 'United Kingdom'), ('winnerName', 'Liverpool F.C.')]
[('edition', '1983–84 European Cup'), ('countryName', 'United Kingdom'), ('winnerName', 'Liverpool F.C.')]
[('edition', '2007–08 UEFA Champions League'), ('countryName', 'United Kingdom'), ('winnerName', 'Manchester United F.C.')]
[('edition', 'UEFA Champions League 2010-11'), ('countryName', 'United Kingdom'), ('winnerNa

In [129]:
# write your queries
# UEFA Champions League (Q20972117)
# sports season of league or competition (P3450)
# [('obj', 'http://www.wikidata.org/entity/Q18756'), ('name', 'UEFA Champions League')]
# let's search for the UEFA winner (P1346)
# try to find country name
queryString = """
SELECT DISTINCT ?edition ?countryName ?winnerName
WHERE {
   # bind something
   ?s wdt:P3450 wd:Q18756;
       wdt:P1346 ?winner .
   ?winner wdt:P17 ?country .
   # get the label
   ?s sc:name ?edition .
   ?country sc:name ?countryName.
   ?winner sc:name ?winnerName
    FILTER (regex(?edition,"UEFA Champions League"))
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('edition', '1992–93 UEFA Champions League'), ('countryName', 'France'), ('winnerName', 'Olympique de Marseille')]
[('edition', '1998–99 UEFA Champions League'), ('countryName', 'United Kingdom'), ('winnerName', 'Manchester United F.C.')]
[('edition', '2018–19 UEFA Champions League'), ('countryName', 'United Kingdom'), ('winnerName', 'Liverpool F.C.')]
[('edition', '2011–12 UEFA Champions League'), ('countryName', 'United Kingdom'), ('winnerName', 'Chelsea F.C.')]
[('edition', '2007–08 UEFA Champions League'), ('countryName', 'United Kingdom'), ('winnerName', 'Manchester United F.C.')]
[('edition', '2004–05 UEFA Champions League'), ('countryName', 'United Kingdom'), ('winnerName', 'Liverpool F.C.')]
[('edition', '2020–21 UEFA Champions League'), ('countryName', 'United Kingdom'), ('winnerName', 'Chelsea F.C.')]
[('edition', 'UEFA Champions League 2010-11'), ('countryName', 'United Kingdom'), ('winnerName', 'Manchester United F.C.')]
[('edition', '1996–97 UEFA Champions League'

In [131]:
# write your queries
# UEFA Champions League (Q20972117)
# sports season of league or competition (P3450)
# [('obj', 'http://www.wikidata.org/entity/Q18756'), ('name', 'UEFA Champions League')]
# let's search for the UEFA winner (P1346)
# try to find country name
queryString = """
SELECT DISTINCT ?country ?countryName (COUNT(?winner) as ?win)
WHERE {
   # bind something
   ?s wdt:P3450 wd:Q18756;
       wdt:P1346 ?winner .
   ?winner wdt:P17 ?country .
   # get the label
   ?s sc:name ?edition .
   ?country sc:name ?countryName.
   ?winner sc:name ?winnerName
    #FILTER (regex(?edition,"UEFA Champions League"))
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q45'), ('countryName', 'Portugal'), ('win', '4')]
[('country', 'http://www.wikidata.org/entity/Q403'), ('countryName', 'Serbia'), ('win', '1')]
[('country', 'http://www.wikidata.org/entity/Q218'), ('countryName', 'Romania'), ('win', '1')]
[('country', 'http://www.wikidata.org/entity/Q55'), ('countryName', 'Netherlands'), ('win', '6')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('win', '12')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom'), ('win', '16')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('win', '8')]
[('country', 'http://www.wikidata.org/entity/Q29999'), ('countryName', 'Kingdom of the Netherlands'), ('win', '1')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('win', '1')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain'), ('win', '17')]
10


In [132]:
# write your queries
# UEFA Champions League (Q20972117)
# sports season of league or competition (P3450)
# [('obj', 'http://www.wikidata.org/entity/Q18756'), ('name', 'UEFA Champions League')]
# let's search for the UEFA winner (P1346)
# try to find country name
queryString = """
SELECT DISTINCT ?country ?countryName (COUNT(?winner) as ?win)
WHERE {
   # bind something
   ?s wdt:P3450 wd:Q18756;
       wdt:P1346 ?winner .
   ?winner wdt:P17 ?country .
   # get the label
   ?s sc:name ?edition .
   ?country sc:name ?countryName.
   ?winner sc:name ?winnerName
   FILTER (regex(?edition,"UEFA Champions League"))
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q45'), ('countryName', 'Portugal'), ('win', '1')]
[('country', 'http://www.wikidata.org/entity/Q55'), ('countryName', 'Netherlands'), ('win', '1')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('win', '5')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom'), ('win', '7')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('win', '4')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('win', '1')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain'), ('win', '10')]
7


In [143]:
# write your queries
# UEFA Champions League (Q20972117)
# sports season of league or competition (P3450)
# [('obj', 'http://www.wikidata.org/entity/Q18756'), ('name', 'UEFA Champions League')]
# let's search for the UEFA winner (P1346)
# try to find country name
queryString = """
SELECT DISTINCT ?country ?countryName (COUNT(?winner) as ?win)
WHERE {
   # bind something
   ?s wdt:P3450 wd:Q18756;
       wdt:P1346 ?winner .
   ?winner wdt:P17 ?country .
   # get the label
   ?s sc:name ?edition .
   ?country sc:name ?countryName.
   ?winner sc:name ?winnerName
   #FILTER (regex(?edition,"UEFA Champions League"))
}
GROUP BY ?country ?countryName
HAVING (COUNT(?winner) >= 2)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('win', '12')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('win', '8')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain'), ('win', '17')]
[('country', 'http://www.wikidata.org/entity/Q45'), ('countryName', 'Portugal'), ('win', '4')]
[('country', 'http://www.wikidata.org/entity/Q55'), ('countryName', 'Netherlands'), ('win', '6')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom'), ('win', '16')]
6


Final query for this task

In [142]:
# write your final query
# write your queries
# UEFA Champions League (Q20972117)
# sports season of league or competition (P3450)
# [('obj', 'http://www.wikidata.org/entity/Q18756'), ('name', 'UEFA Champions League')]
# let's search for the UEFA winner (P1346)
# try to find country name
queryString = """
SELECT DISTINCT ?country ?countryName (COUNT(?winner) as ?win)
WHERE {
   # bind something
   ?s wdt:P3450 wd:Q18756;
       wdt:P1346 ?winner .
   ?winner wdt:P17 ?country .
   # get the label
   ?s sc:name ?edition .
   ?country sc:name ?countryName.
   ?winner sc:name ?winnerName
   FILTER (regex(?edition,"UEFA Champions League"))
}
GROUP BY ?country ?countryName
HAVING (COUNT(?winner) >= 2)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('win', '5')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('win', '4')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain'), ('win', '10')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom'), ('win', '7')]
4


## Task 6
Return for each european country the top level league (the result set must be a list of country IRI and label, and league IRI and label).

In [65]:
# write your queries
#lets search for top italian football league
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pname
WHERE {
   # bind something
   ?s ?p wd:Q15804 .
   # get the label
   ?p sc:name ?pname.
   ?s sc:name ?sName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q45901'), ('sName', 'Thierry Henry'), ('p', 'http://www.wikidata.org/prop/direct/P118'), ('pname', 'league')]
[('s', 'http://www.wikidata.org/entity/Q223724'), ('sName', '2009–10 Serie A'), ('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pname', 'sports season of league or competition')]
[('s', 'http://www.wikidata.org/entity/Q240669'), ('sName', '2011–12 Serie A'), ('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pname', 'sports season of league or competition')]
[('s', 'http://www.wikidata.org/entity/Q276204'), ('sName', '2010–11 Serie A'), ('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pname', 'sports season of league or competition')]
[('s', 'http://www.wikidata.org/entity/Q381013'), ('sName', '2007–08 Serie A'), ('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pname', 'sports season of league or competition')]
[('s', 'http://www.wikidata.org/entity/Q643978'), ('sName', '2005–06 Serie A'), ('p', 'http://www.wikid

In [66]:
# write your queries
#lets search for top italian football league
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pname
WHERE {
   # bind something
   ?s ?p wd:Q15804 .
   # get the label
   ?p sc:name ?pname.
   ?s sc:name ?sName.
   filter(regex(?pname,"top level"))
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [67]:
# write your queries
#lets search for top italian football league
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pname
WHERE {
   # bind something
   ?s ?p wd:Q15804 .
   # get the label
   ?p sc:name ?pname.
   ?s sc:name ?sName.
   filter(regex(?pname,"top"))
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q6807104'), ('sName', 'Category:Serie A'), ('p', 'http://www.wikidata.org/prop/direct/P301'), ('pname', "category's main topic")]
[('s', 'http://www.wikidata.org/entity/Q7195347'), ('sName', 'Category:Serie A seasons'), ('p', 'http://www.wikidata.org/prop/direct/P971'), ('pname', 'category combines topics')]
[('s', 'http://www.wikidata.org/entity/Q10956030'), ('sName', 'Template:Serie A seasons'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pname', 'template has topic')]
[('s', 'http://www.wikidata.org/entity/Q7119823'), ('sName', 'Category:Serie A players'), ('p', 'http://www.wikidata.org/prop/direct/P971'), ('pname', 'category combines topics')]
4


In [68]:
# write your queries
#lets search for top italian football league
# country (Q6256)
# association football league (Q15991303)
queryString = """
SELECT DISTINCT ?europeanCountry ?countryName ?league ?leagueName
WHERE {
   # bind something
   ?europeanCountry wdt:P31 wd:Q6256 ;
               wdt:P30 wd:Q46 .
   ?league wdt:P17 ?europeanCountries ;
            wdt:P31 wd:Q15991303 .
   # get the label
   ?europeanCountry sc:name ?countryName .
   ?league sc:name ?leagueName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('europeanCountry', 'http://www.wikidata.org/entity/Q229'), ('countryName', 'Cyprus'), ('league', 'http://www.wikidata.org/entity/Q304228'), ('leagueName', 'Uzbekistan Pro League')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q229'), ('countryName', 'Cyprus'), ('league', 'http://www.wikidata.org/entity/Q2950655'), ('leagueName', 'Ligue 2')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q229'), ('countryName', 'Cyprus'), ('league', 'http://www.wikidata.org/entity/Q2951924'), ('leagueName', "Moldovan women's football championship")]
[('europeanCountry', 'http://www.wikidata.org/entity/Q229'), ('countryName', 'Cyprus'), ('league', 'http://www.wikidata.org/entity/Q973312'), ('leagueName', 'Championnat National 3')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q229'), ('countryName', 'Cyprus'), ('league', 'http://www.wikidata.org/entity/Q2792041'), ('leagueName', '3. divisjon')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q229'), ('countryName', 'Cy

In [71]:
# write your queries
#lets search for top italian football league
# country (Q6256)
# association football league (Q15991303)
queryString = """
SELECT DISTINCT ?country ?countryName ?league ?leagueName
WHERE {
   # bind something
   ?country wdt:P31 wd:Q6256 ;
               wdt:P30 wd:Q46 .
   ?league wdt:P17 ?country ;
            wdt:P31 wd:Q15991303 .
   # get the label
   ?country sc:name ?countryName .
   ?league sc:name ?leagueName .
   MINUS {?someLeague wdt:P2499 ?league}.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q973312'), ('leagueName', 'Championnat National 3')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q3240443'), ('leagueName', 'Ligue de la Méditerranée de football')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q891072'), ('leagueName', 'French Guiana Championnat National')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q2261921'), ('leagueName', 'Réunion Premier League')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q2951169'), ('leagueName', 'Division 2 Féminine')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('leag

In [72]:
# write your queries
#lets search for top italian football league
# country (Q6256)
# association football league (Q15991303)
queryString = """
SELECT DISTINCT ?country ?countryName ?league ?leagueName
WHERE {
   # bind something
   ?country wdt:P31 wd:Q6256 ;
               wdt:P30 wd:Q46 .
   ?league wdt:P17 ?country ;
            wdt:P31 wd:Q15991303 .
   # get the label
   ?country sc:name ?countryName .
   ?league sc:name ?leagueName .
   MINUS {?league wdt:P2499 ?fewLeague}.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q3240443'), ('leagueName', 'Ligue de la Méditerranée de football')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q891072'), ('leagueName', 'French Guiana Championnat National')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q600302'), ('leagueName', 'New Caledonia Division Honneur')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q2261921'), ('leagueName', 'Réunion Premier League')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', 'http://www.wikidata.org/entity/Q13394'), ('leagueName', 'Ligue 1')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('league', '

In [6]:
# write your queries
#lets search for top italian football league
# country (Q6256)
# association football league (Q15991303)
# sports league level (P3983)
# retrieve all the leagues with sports league level 1 of european countries
queryString = """
SELECT DISTINCT ?country ?countryName ?league ?leagueName
WHERE {
   # bind something
   ?country wdt:P31 wd:Q6256 ;
               wdt:P30 wd:Q46 .
   ?league wdt:P17 ?country ;
            wdt:P31 wd:Q15991303;
            wdt:P3983 1 .
   # get the label
   ?country sc:name ?countryName .
   ?league sc:name ?leagueName .
   MINUS {?league wdt:P2499 ?fewLeague}.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q214'), ('countryName', 'Slovakia'), ('league', 'http://www.wikidata.org/entity/Q238113'), ('leagueName', 'Slovak Super Liga')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('countryName', 'Denmark'), ('league', 'http://www.wikidata.org/entity/Q739479'), ('leagueName', 'Faroe Islands Premier League')]
[('country', 'http://www.wikidata.org/entity/Q33'), ('countryName', 'Finland'), ('league', 'http://www.wikidata.org/entity/Q220875'), ('leagueName', 'Veikkausliiga')]
[('country', 'http://www.wikidata.org/entity/Q45'), ('countryName', 'Portugal'), ('league', 'http://www.wikidata.org/entity/Q182994'), ('leagueName', 'Liga Portugal')]
[('country', 'http://www.wikidata.org/entity/Q34'), ('countryName', 'Sweden'), ('league', 'http://www.wikidata.org/entity/Q202243'), ('leagueName', 'Allsvenskan')]
[('country', 'http://www.wikidata.org/entity/Q36'), ('countryName', 'Poland'), ('league', 'http://www.wikidata.org/entity/Q44763'), ('leag

In [77]:
# write your queries
# country (Q6256)
# association football league (Q15991303)
# sports league level (P3983)
# retrieve all the leagues with sports league level 1 of european countries
queryString = """
SELECT DISTINCT ?country ?countryName ?league ?leagueName
WHERE {
   # bind something
   ?country wdt:P31 wd:Q6256 ;
               wdt:P30 wd:Q46 .
   ?league wdt:P17 ?country ;
            wdt:P31 wd:Q15991303;
            wdt:P3983 1 .
   # get the label
   ?country sc:name ?countryName .
   ?league sc:name ?leagueName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q214'), ('countryName', 'Slovakia'), ('league', 'http://www.wikidata.org/entity/Q238113'), ('leagueName', 'Slovak Super Liga')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('countryName', 'Denmark'), ('league', 'http://www.wikidata.org/entity/Q739479'), ('leagueName', 'Faroe Islands Premier League')]
[('country', 'http://www.wikidata.org/entity/Q33'), ('countryName', 'Finland'), ('league', 'http://www.wikidata.org/entity/Q220875'), ('leagueName', 'Veikkausliiga')]
[('country', 'http://www.wikidata.org/entity/Q45'), ('countryName', 'Portugal'), ('league', 'http://www.wikidata.org/entity/Q182994'), ('leagueName', 'Liga Portugal')]
[('country', 'http://www.wikidata.org/entity/Q34'), ('countryName', 'Sweden'), ('league', 'http://www.wikidata.org/entity/Q202243'), ('leagueName', 'Allsvenskan')]
[('country', 'http://www.wikidata.org/entity/Q36'), ('countryName', 'Poland'), ('league', 'http://www.wikidata.org/entity/Q44763'), ('leag

Final query for this task

In [7]:
# write your final query
# country (Q6256)
# association football league (Q15991303)
# sports league level (P3983)
# retrieve all the leagues with sports league level 1 of european countries
queryString = """
SELECT DISTINCT ?country ?countryName ?league ?leagueName
WHERE {
   # bind something
   ?country wdt:P31 wd:Q6256 ;
               wdt:P30 wd:Q46 .
   ?league wdt:P17 ?country ;
            wdt:P31 wd:Q15991303;
            wdt:P3983 1 .
   # get the label
   ?country sc:name ?countryName .
   ?league sc:name ?leagueName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q214'), ('countryName', 'Slovakia'), ('league', 'http://www.wikidata.org/entity/Q238113'), ('leagueName', 'Slovak Super Liga')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('countryName', 'Denmark'), ('league', 'http://www.wikidata.org/entity/Q739479'), ('leagueName', 'Faroe Islands Premier League')]
[('country', 'http://www.wikidata.org/entity/Q33'), ('countryName', 'Finland'), ('league', 'http://www.wikidata.org/entity/Q220875'), ('leagueName', 'Veikkausliiga')]
[('country', 'http://www.wikidata.org/entity/Q45'), ('countryName', 'Portugal'), ('league', 'http://www.wikidata.org/entity/Q182994'), ('leagueName', 'Liga Portugal')]
[('country', 'http://www.wikidata.org/entity/Q34'), ('countryName', 'Sweden'), ('league', 'http://www.wikidata.org/entity/Q202243'), ('leagueName', 'Allsvenskan')]
[('country', 'http://www.wikidata.org/entity/Q36'), ('countryName', 'Poland'), ('league', 'http://www.wikidata.org/entity/Q44763'), ('leag

## Task 7
Identify the BGP of the FIFA World Cup

In [144]:
# write your queries
# FIFA Club World Cup (Q223366)
queryString = """
SELECT DISTINCT ?p ?pname ?obj ?objname
WHERE {
   # bind something
   wd:Q223366 ?p ?obj .
   # get the label
   ?p sc:name ?pname.
   ?obj sc:name ?objname.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pname', 'organizer'), ('obj', 'http://www.wikidata.org/entity/Q253414'), ('objname', 'FIFA')]
[('p', 'http://www.wikidata.org/prop/direct/P4794'), ('pname', 'season starts'), ('obj', 'http://www.wikidata.org/entity/Q126'), ('objname', 'December')]
[('p', 'http://www.wikidata.org/prop/direct/P2922'), ('pname', 'month of the year'), ('obj', 'http://www.wikidata.org/entity/Q126'), ('objname', 'December')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q18608583'), ('objname', 'recurring sporting event')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q2736'), ('objname', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q15789'), ('objname', 'FC Bayern Munich')]
[('p', 'http://www.wikidata.org/prop/direct/P2

In [149]:
# write your queries
# FIFA Club World Cup (Q223366)
queryString = """
SELECT DISTINCT ?obj ?objname
WHERE {
   # bind something
   wd:Q223366 wdt:P3450 ?obj .
   # get the label
   #?p sc:name ?pname.
   ?obj sc:name ?objname.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [150]:
# write your queries
# FIFA Club World Cup (Q223366)
queryString = """
SELECT DISTINCT ?p ?objname
WHERE {
   # bind something
   wd:Q223366 ?p ?obj .
   # get the label
   #?p sc:name ?pname.
   ?p sc:name ?objname.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('objname', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('objname', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1350'), ('objname', 'number of matches played/races/starts')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('objname', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('objname', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2257'), ('objname', 'event interval')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('objname', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P2922'), ('objname', 'month of the year')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('objname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('objname', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('objname', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P4794'), (

In [151]:
# write your queries
# FIFA Club World Cup (Q223366)
queryString = """
SELECT DISTINCT ?obj ?objname
WHERE {
   # bind something
   wd:Q223366 wdt:P641 ?obj .
   # get the label
   #?p sc:name ?pname.
   ?obj sc:name ?objname.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q2736'), ('objname', 'association football')]
1


In [153]:
# write your queries
# FIFA Club World Cup (Q223366)
# 2018 FIFA World Cup Q170645
queryString = """
SELECT DISTINCT ?p ?objname
WHERE {
   # bind something
   wd:Q170645 ?p ?obj .
   # get the label
   ?p sc:name ?pname.
   ?obj sc:name ?objname.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3967'), ('objname', '2018 FIFA World Cup Final')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('objname', 'FIFA')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('objname', 'Luzhniki Stadium')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('objname', 'Russia')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('objname', 'sports season')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('objname', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('objname', 'France national association football team')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('objname', "men's association football")]
[('p', 'http://www.wikidata.org/prop/direct/P3279'), ('objname', 'Harry Kane')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('objname', 'Otkrytie Bank Arena')]
[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('objname', 'FIFA World Cup')]
[('p', 'http://www.wikidata.org/prop

In [154]:
# write your queries
# [('p', 'http://www.wikidata.org/prop/direct/P3450'), ('objname', 'FIFA World Cup')]
# 2018 FIFA World Cup Q170645
queryString = """
SELECT DISTINCT ?obj ?objname
WHERE {
   # bind something
   wd:Q170645 wdt:P3450 ?obj .
   # get the label
   #?p sc:name ?pname.
   ?obj sc:name ?objname.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q19317'), ('objname', 'FIFA World Cup')]
1


Final query for this task

In [155]:
# write your queries
# # write your final query
# [('p', 'http://www.wikidata.org/prop/direct/P3450'), ('objname', 'FIFA World Cup')]
# 2018 FIFA World Cup Q170645
queryString = """
SELECT DISTINCT ?obj ?objname
WHERE {
   # bind something
   wd:Q170645 wdt:P3450 ?obj .
   # get the label
   #?p sc:name ?pname.
   ?obj sc:name ?objname.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q19317'), ('objname', 'FIFA World Cup')]
1


## Task 8
Consider countries who participated at the FIFA World Cup more than 10 times. Return these countries with the number of apparitions (the result set must be triples of country IRI, label and #apparitions).

In [159]:
# write your queries
# let's search for the fifa world cup editions
queryString = """
SELECT DISTINCT ?s ?editionName
WHERE {
   # bind something
   ?s wdt:P3450 wd:Q19317 .
   # get the label
   ?s sc:name ?editionName.
}
ORDER BY ?editionName
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q48432'), ('editionName', '1930 FIFA World Cup')]
[('s', 'http://www.wikidata.org/entity/Q1477177'), ('editionName', '1934 FIFA World Cup')]
[('s', 'http://www.wikidata.org/entity/Q131591'), ('editionName', '1938 FIFA World Cup')]
[('s', 'http://www.wikidata.org/entity/Q203851'), ('editionName', '1942 FIFA World Cup')]
[('s', 'http://www.wikidata.org/entity/Q65273310'), ('editionName', '1946 FIFA World Cup')]
[('s', 'http://www.wikidata.org/entity/Q132515'), ('editionName', '1950 FIFA World Cup')]
[('s', 'http://www.wikidata.org/entity/Q131075'), ('editionName', '1954 FIFA World Cup')]
[('s', 'http://www.wikidata.org/entity/Q132533'), ('editionName', '1958 FIFA World Cup')]
[('s', 'http://www.wikidata.org/entity/Q160813'), ('editionName', '1962 FIFA World Cup')]
[('s', 'http://www.wikidata.org/entity/Q134202'), ('editionName', '1966 FIFA World Cup')]
[('s', 'http://www.wikidata.org/entity/Q132664'), ('editionName', '1970 FIFA World Cup')]


In [165]:
# write your queries
# let's search for the fifa world cup editions
# 2018 FIFA World Cup (Q170645)
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q170645 ?p ?obj .
   # get the label
   ?p sc:name ?pname .
   ?obj sc:name ?objName.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P3967'), ('obj', 'http://www.wikidata.org/entity/Q31043671'), ('objName', '2018 FIFA World Cup Final')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('obj', 'http://www.wikidata.org/entity/Q253414'), ('objName', 'FIFA')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('obj', 'http://www.wikidata.org/entity/Q202163'), ('objName', 'Luzhniki Stadium')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('obj', 'http://www.wikidata.org/entity/Q159'), ('objName', 'Russia')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('obj', 'http://www.wikidata.org/entity/Q27020041'), ('objName', 'sports season')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('obj', 'http://www.wikidata.org/entity/Q2736'), ('objName', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('obj', 'http://www.wikidata.org/entity/Q47774'), ('objName', 'France national association football team')]
[('p', 'http://www.wikidata.o

In [7]:
# write your queries
# let's search for the fifa world cup editions
# 2018 FIFA World Cup (Q170645)
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q170645 ?p ?obj .
   
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P3967'), ('pName', 'final event'), ('obj', 'http://www.wikidata.org/entity/Q31043671'), ('objName', '2018 FIFA World Cup Final')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('obj', 'http://www.wikidata.org/entity/Q253414'), ('objName', 'FIFA')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('obj', 'http://www.wikidata.org/entity/Q202163'), ('objName', 'Luzhniki Stadium')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q159'), ('objName', 'Russia')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q27020041'), ('objName', 'sports season')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q2736'), ('objName', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P1346

In [9]:
# write your queries
# let's search for the fifa world cup editions
# 2018 FIFA World Cup (Q170645)
# [('p', 'http://www.wikidata.org/prop/direct/P1923'), ('pName', 'participating team')
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # bind something
   wd:Q170645 wdt:P1923 ?obj .
   
   # get the label
   #?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('obj', 'http://www.wikidata.org/entity/Q45101586'), ('objName', 'Australia at the 2018 FIFA World Cup')]
[('obj', 'http://www.wikidata.org/entity/Q43196508'), ('objName', 'Uruguay at the 2018 FIFA World Cup\u200e')]
[('obj', 'http://www.wikidata.org/entity/Q43196538'), ('objName', 'Brazil at the 2018 FIFA World Cup\u200e')]
[('obj', 'http://www.wikidata.org/entity/Q43122212'), ('objName', 'Iceland at the 2018 FIFA World Cup')]
[('obj', 'http://www.wikidata.org/entity/Q45905387'), ('objName', 'Croatia at the 2018 FIFA World Cup')]
[('obj', 'http://www.wikidata.org/entity/Q43197686'), ('objName', 'Panama at the 2018 FIFA World Cup')]
[('obj', 'http://www.wikidata.org/entity/Q43249937'), ('objName', 'France at the 2018 FIFA World Cup\u200e')]
[('obj', 'http://www.wikidata.org/entity/Q44959733'), ('objName', 'Egypt at the 2018 FIFA World Cup\u200e')]
[('obj', 'http://www.wikidata.org/entity/Q43122091'), ('objName', 'Germany at the 2018 FIFA World Cup')]
[('obj', 'http://www.wikid

In [12]:
# write your queries
# let's search for the fifa world cup editions
# 2018 FIFA World Cup (Q170645)
# [('p', 'http://www.wikidata.org/prop/direct/P1923'), ('pName', 'participating team')
queryString = """
SELECT DISTINCT ?country ?countryName
WHERE {
   # bind something
   wd:Q170645 wdt:P1923 ?pertTeam .
   
   ?pertTeam wdt:P17 ?country .
   
   # get the label
   #?p sc:name ?pName.
   #?obj sc:name ?objName.
   ?country sc:name ?countryName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('country', 'http://www.wikidata.org/entity/Q1041'), ('countryName', 'Senegal')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain')]
[('country', 'http://www.wikidata.org/entity/Q34'), ('countryName', 'Sweden')]
[('country', 'http://www.wikidata.org/entity/Q408'), ('countryName', 'Australia')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('countryName', 'Argentina')]
[('country', 'http://www.wikidata.org/entity/Q419'), ('countryName', 'Peru')]
[('country', 'http://www.wikidata.org/entity/Q45'), ('countryName', 'Portugal')]
[('country', 'http://www.wikidata.org/entity/Q739'), ('countryName', 'Colombia')]
[('country', 'http://www.wikidata.org/entity/Q77'), ('countryName', 'Uruguay')]
[('country', 'http://www.wikidata.org/entity/Q79'), ('countryName', 'Egypt')]
[('country', 'http:/

In [13]:
# write your queries
# let's search for the fifa world cup editions
# 2018 FIFA World Cup (Q170645)
# [('p', 'http://www.wikidata.org/prop/direct/P1923'), ('pName', 'participating team')
# country for sport (P1532)
queryString = """
SELECT DISTINCT ?country ?countryName
WHERE {
   # bind something
   wd:Q170645 wdt:P1923 ?pertTeam .
   
   ?pertTeam wdt:P1532 ?country .
   
   # get the label
   #?p sc:name ?pName.
   #?obj sc:name ?objName.
   ?country sc:name ?countryName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('country', 'http://www.wikidata.org/entity/Q1028'), ('countryName', 'Morocco')]
[('country', 'http://www.wikidata.org/entity/Q1033'), ('countryName', 'Nigeria')]
[('country', 'http://www.wikidata.org/entity/Q1041'), ('countryName', 'Senegal')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('countryName', 'Russia')]
[('country', 'http://www.wikidata.org/entity/Q17'), ('countryName', 'Japan')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q189'), ('countryName', 'Iceland')]
[('country', 'http://www.wikidata.org/entity/Q21'), ('countryName', 'England')]
[('country', 'http://www.wikidata.org/entity/Q224'), ('countryName', 'Croatia')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain')]
[('country', 'http://www.wi

In [14]:
# write your queries
# 2018 FIFA World Cup team (Q54812340)
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q54812340 ?p ?obj .
   
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?objName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in'), ('obj', 'http://www.wikidata.org/entity/Q170645'), ('objName', '2018 FIFA World Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q2736'), ('objName', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q46135307'), ('objName', 'nation at sport competition')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q1477115'), ('objName', 'FIFA World Cup team')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('obj', 'http://www.wikidata.org/entity/Q20436354'), ('objName', '2014 FIFA World Cup team')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q31529939'), ('objName', 'Cat

In [15]:
# write your queries
# let's find who perticipate in 2018 fifa world cup
# 2018 FIFA World Cup team (Q54812340)
# country for sport (P1532)
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # bind something
   wd:Q54812340 ?s ?p .
   
   # get the label
   ?p sc:name ?pName.
   ?s sc:name ?sName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/prop/direct/P1344'), ('sName', 'participant in'), ('p', 'http://www.wikidata.org/entity/Q170645'), ('pName', '2018 FIFA World Cup')]
[('s', 'http://www.wikidata.org/prop/direct/P155'), ('sName', 'follows'), ('p', 'http://www.wikidata.org/entity/Q20436354'), ('pName', '2014 FIFA World Cup team')]
[('s', 'http://www.wikidata.org/prop/direct/P2354'), ('sName', 'has list'), ('p', 'http://www.wikidata.org/entity/Q45056532'), ('pName', '2018 FIFA World Cup squads')]
[('s', 'http://www.wikidata.org/prop/direct/P279'), ('sName', 'subclass of'), ('p', 'http://www.wikidata.org/entity/Q1477115'), ('pName', 'FIFA World Cup team')]
[('s', 'http://www.wikidata.org/prop/direct/P31'), ('sName', 'instance of'), ('p', 'http://www.wikidata.org/entity/Q46135307'), ('pName', 'nation at sport competition')]
[('s', 'http://www.wikidata.org/prop/direct/P641'), ('sName', 'sport'), ('p', 'http://www.wikidata.org/entity/Q2736'), ('pName', 'association football')]
[('s', 'h

In [16]:
# write your queries
# let's find who perticipate in 2018 world cup
# 2018 FIFA World Cup team (Q54812340)
# country for sport (P1532)
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # bind something
   ?s ?p wd:Q54812340 .
   
   # get the label
   ?p sc:name ?pName.
   ?s sc:name ?sName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q20436354'), ('sName', '2014 FIFA World Cup team'), ('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by')]
[('s', 'http://www.wikidata.org/entity/Q45101586'), ('sName', 'Australia at the 2018 FIFA World Cup'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q43196508'), ('sName', 'Uruguay at the 2018 FIFA World Cup\u200e'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q43196538'), ('sName', 'Brazil at the 2018 FIFA World Cup\u200e'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q43122212'), ('sName', 'Iceland at the 2018 FIFA World Cup'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q45905387'), ('sName', 'Croatia at the 2018 FIFA World Cup'), ('

In [20]:
# write your queries
# let's search for the fifa world cup editions
# 2018 FIFA World Cup (Q170645)
# [('p', 'http://www.wikidata.org/prop/direct/P1923'), ('pName', 'participating team')
# country for sport (P1532)
queryString = """
SELECT DISTINCT (COUNT(?country) as ?perticipation) ?country ?countryName
WHERE {
   # bind something
   wd:Q170645 wdt:P1923 ?pertTeam .
   
   ?pertTeam wdt:P1532 ?country .
   
   # get the label
   #?p sc:name ?pName.
   #?obj sc:name ?objName.
   ?country sc:name ?countryName .
}
GROUP BY ?country ?countryName
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('perticipation', '1'), ('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany')]
[('perticipation', '1'), ('country', 'http://www.wikidata.org/entity/Q189'), ('countryName', 'Iceland')]
[('perticipation', '1'), ('country', 'http://www.wikidata.org/entity/Q1041'), ('countryName', 'Senegal')]
[('perticipation', '1'), ('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil')]
[('perticipation', '1'), ('country', 'http://www.wikidata.org/entity/Q31'), ('countryName', 'Belgium')]
[('perticipation', '1'), ('country', 'http://www.wikidata.org/entity/Q34'), ('countryName', 'Sweden')]
[('perticipation', '1'), ('country', 'http://www.wikidata.org/entity/Q403'), ('countryName', 'Serbia')]
[('perticipation', '1'), ('country', 'http://www.wikidata.org/entity/Q419'), ('countryName', 'Peru')]
[('perticipation', '1'), ('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain')]
[('perticipation', '1'), ('country', 'http://www.wikidata

In [23]:
# write your queries
# let's search for the fifa world cup editions
# 2018 FIFA World Cup (Q170645)
# [('p', 'http://www.wikidata.org/prop/direct/P1923'), ('pName', 'participating team')
# [('p', 'http://www.wikidata.org/prop/direct/P3450'), ('objname', 'FIFA World Cup')]
# country for sport (P1532)
queryString = """
SELECT DISTINCT (COUNT(?country) as ?perticipation) ?country ?countryName
WHERE {
   # bind something
   ?edi wd:Q19317 wdt:P3450;
       wdt:P1923 ?partTeam .
   
   ?pertTeam wdt:P1532 ?country .
   
   # get the label
   #?p sc:name ?pName.
   #?obj sc:name ?objName.
   ?country sc:name ?countryName .
}
GROUP BY ?country ?countryName
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
Empty


In [28]:
# write your queries
# let's search for the fifa world cup editions
# 2018 FIFA World Cup (Q170645)
# [('p', 'http://www.wikidata.org/prop/direct/P1923'), ('pName', 'participating team')
# [('p', 'http://www.wikidata.org/prop/direct/P3450'), ('objname', 'FIFA World Cup')]
# country for sport (P1532)
queryString = """
SELECT DISTINCT ?country ?countryName (COUNT(?country) as ?perticipation)
WHERE {
   # bind something
   ?edi wdt:P3450 wd:Q19317;
       wdt:P1923 ?partTeam .
   
   ?partTeam wdt:P1532 ?country .
   
   # get the label
   #?p sc:name ?pName.
   #?obj sc:name ?objName.
   ?country sc:name ?countryName .
}
GROUP BY ?country ?countryName
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('country', 'http://www.wikidata.org/entity/Q215'), ('countryName', 'Slovenia'), ('perticipation', '2')]
[('country', 'http://www.wikidata.org/entity/Q218'), ('countryName', 'Romania'), ('perticipation', '7')]
[('country', 'http://www.wikidata.org/entity/Q28'), ('countryName', 'Hungary'), ('perticipation', '9')]
[('country', 'http://www.wikidata.org/entity/Q754'), ('countryName', 'Trinidad and Tobago'), ('perticipation', '1')]
[('country', 'http://www.wikidata.org/entity/Q974'), ('countryName', 'Democratic Republic of the Congo'), ('perticipation', '1')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('countryName', 'Canada'), ('perticipation', '1')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('perticipation', '18')]
[('country', 'http://www.wikidata.org/entity/Q664'), ('countryName', 'New Zealand'), ('perticipation', '2')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('perticipation', '19')]
[('countr

In [30]:
# write your queries
# [('p', 'http://www.wikidata.org/prop/direct/P1923'), ('pName', 'participating team')
# [('p', 'http://www.wikidata.org/prop/direct/P3450'), ('objname', 'FIFA World Cup')]
# [('obj', 'http://www.wikidata.org/entity/Q19317'), ('objname', 'FIFA World Cup')]
# country for sport (P1532)
queryString = """
SELECT DISTINCT ?country ?countryName (COUNT(?country) as ?perticipation)
WHERE {
   # bind something
   ?s wdt:P3450 wd:Q19317;
       wdt:P1923 ?partTeam .
   
   ?partTeam wdt:P1532 ?country .
   
   # get the label
   #?p sc:name ?pName.
   #?obj sc:name ?objName.
   ?country sc:name ?countryName .
}
GROUP BY ?country ?countryName
HAVING (COUNT(?country)>=10)
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('perticipation', '18')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('perticipation', '19')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil'), ('perticipation', '21')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('countryName', 'Belgium'), ('perticipation', '13')]
[('country', 'http://www.wikidata.org/entity/Q34'), ('countryName', 'Sweden'), ('perticipation', '12')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain'), ('perticipation', '15')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('countryName', 'Argentina'), ('perticipation', '17')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('perticipation', '15')]
[('country', 'http://www.wikidata.org/entity/Q884'), ('countryName', 'South Korea'), ('perticipation', '10')]
[('country', 'http://www.wikidata.org/entit

Final query for this task

In [31]:
# # write your final query
# [('p', 'http://www.wikidata.org/prop/direct/P1923'), ('pName', 'participating team')
# [('p', 'http://www.wikidata.org/prop/direct/P3450'), ('objname', 'FIFA World Cup')]
# [('obj', 'http://www.wikidata.org/entity/Q19317'), ('objname', 'FIFA World Cup')]
# country for sport (P1532)
queryString = """
SELECT DISTINCT ?country ?countryName (COUNT(?country) as ?perticipation)
WHERE {
   # bind something
   ?s wdt:P3450 wd:Q19317;
       wdt:P1923 ?partTeam .
   
   ?partTeam wdt:P1532 ?country .
   
   # get the label
   #?p sc:name ?pName.
   #?obj sc:name ?objName.
   ?country sc:name ?countryName .
}
GROUP BY ?country ?countryName
HAVING (COUNT(?country)>=10)
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('perticipation', '18')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('perticipation', '19')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil'), ('perticipation', '21')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('countryName', 'Belgium'), ('perticipation', '13')]
[('country', 'http://www.wikidata.org/entity/Q34'), ('countryName', 'Sweden'), ('perticipation', '12')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain'), ('perticipation', '15')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('countryName', 'Argentina'), ('perticipation', '17')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('perticipation', '15')]
[('country', 'http://www.wikidata.org/entity/Q884'), ('countryName', 'South Korea'), ('perticipation', '10')]
[('country', 'http://www.wikidata.org/entit

## Task 9
Consider the countries that won the FIFA World Cup at least twice. Find the association football clubs of those countries won the UEFA Champions League (the result set must be couples of clubs IRI and label).

In [37]:
# write your queries
# find winner (P1346)
# [('p', 'http://www.wikidata.org/prop/direct/P3450'), ('objname', 'FIFA World Cup')]
# [('obj', 'http://www.wikidata.org/entity/Q19317'), ('objname', 'FIFA World Cup')]
queryString = """
SELECT DISTINCT ?edition ?name
WHERE {
   # bind something
   ?edition wdt:P3450 wd:Q19317 ;
       wdt:P1346 ?national.
   # get the label
   ?edition sc:name ?name.
   ?national sc:name ?nationalName .
}
ORDER BY ?editionName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('edition', 'http://www.wikidata.org/entity/Q46934'), ('name', '1982 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q131591'), ('name', '1938 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q1477177'), ('name', '1934 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q37285'), ('name', '2006 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q132515'), ('name', '1950 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q48432'), ('name', '1930 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q134202'), ('name', '1966 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q170645'), ('name', '2018 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q101730'), ('name', '1998 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q176883'), ('name', '2010 FIFA World Cup')]
[('edition', 'http://www.wikidata.org/entity/Q46943'), ('name', '1978 FIFA World Cup')]
[('edition', 'ht

In [42]:
# write your queries
# find winner (P1346)
# [('p', 'http://www.wikidata.org/prop/direct/P3450'), ('objname', 'FIFA World Cup')] sports season of league or competition (P3450)
# [('obj', 'http://www.wikidata.org/entity/Q19317'), ('objname', 'FIFA World Cup')]
queryString = """
SELECT DISTINCT ?edition ?name ?national ?nationalName
WHERE {
   # bind something
   ?edition wdt:P3450 wd:Q19317 ;
       wdt:P1346 ?national.
   # get the label
   ?edition sc:name ?name.
   ?national sc:name ?nationalName .
}
ORDER BY ?editionName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('edition', 'http://www.wikidata.org/entity/Q46934'), ('name', '1982 FIFA World Cup'), ('national', 'http://www.wikidata.org/entity/Q676899'), ('nationalName', 'Italy national association football team')]
[('edition', 'http://www.wikidata.org/entity/Q131591'), ('name', '1938 FIFA World Cup'), ('national', 'http://www.wikidata.org/entity/Q676899'), ('nationalName', 'Italy national association football team')]
[('edition', 'http://www.wikidata.org/entity/Q1477177'), ('name', '1934 FIFA World Cup'), ('national', 'http://www.wikidata.org/entity/Q676899'), ('nationalName', 'Italy national association football team')]
[('edition', 'http://www.wikidata.org/entity/Q37285'), ('name', '2006 FIFA World Cup'), ('national', 'http://www.wikidata.org/entity/Q676899'), ('nationalName', 'Italy national association football team')]
[('edition', 'http://www.wikidata.org/entity/Q132515'), ('name', '1950 FIFA World Cup'), ('national', 'http://www.wikidata.org/entity/Q134916'), ('nationalName', 'Ur

In [50]:
# write your queries
# find winner (P1346)
# [('p', 'http://www.wikidata.org/prop/direct/P3450'), ('objname', 'FIFA World Cup')] sports season of league or competition (P3450)
# [('obj', 'http://www.wikidata.org/entity/Q19317'), ('objname', 'FIFA World Cup')]
# let's find who win fifa world cup
queryString = """
SELECT DISTINCT ?edition ?editionYear ?country ?winnerCountry
WHERE {
   # bind something
   ?edition wdt:P3450 wd:Q19317 ;
       wdt:P1346 ?winner.
       

   ?winner wdt:P1532 ?country; #find sport country
       wdt:P17 ?country .
   # get the label
   ?edition sc:name ?editionYear.
   ?country sc:name ?winnerCountry .
}
ORDER BY ?editionYear
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('edition', 'http://www.wikidata.org/entity/Q48432'), ('editionYear', '1930 FIFA World Cup'), ('country', 'http://www.wikidata.org/entity/Q77'), ('winnerCountry', 'Uruguay')]
[('edition', 'http://www.wikidata.org/entity/Q1477177'), ('editionYear', '1934 FIFA World Cup'), ('country', 'http://www.wikidata.org/entity/Q38'), ('winnerCountry', 'Italy')]
[('edition', 'http://www.wikidata.org/entity/Q131591'), ('editionYear', '1938 FIFA World Cup'), ('country', 'http://www.wikidata.org/entity/Q38'), ('winnerCountry', 'Italy')]
[('edition', 'http://www.wikidata.org/entity/Q132515'), ('editionYear', '1950 FIFA World Cup'), ('country', 'http://www.wikidata.org/entity/Q77'), ('winnerCountry', 'Uruguay')]
[('edition', 'http://www.wikidata.org/entity/Q131075'), ('editionYear', '1954 FIFA World Cup'), ('country', 'http://www.wikidata.org/entity/Q183'), ('winnerCountry', 'Germany')]
[('edition', 'http://www.wikidata.org/entity/Q132533'), ('editionYear', '1958 FIFA World Cup'), ('country', 'h

In [56]:
# write your queries
# find winner (P1346)
# [('p', 'http://www.wikidata.org/prop/direct/P3450'), ('objname', 'FIFA World Cup')] sports season of league or competition (P3450)
# [('obj', 'http://www.wikidata.org/entity/Q19317'), ('objname', 'FIFA World Cup')]
# let's find who win fifa world cup
# let's find those countries who win world cup 2 or more then 2 times.
queryString = """
SELECT DISTINCT ?country ?winnerCountry
WHERE {
   # bind something
   ?edition wdt:P3450 wd:Q19317 ;
       wdt:P1346 ?winner.
       

   ?winner wdt:P1532 ?country; #find sport country
       wdt:P17 ?country .
   # get the label
   ?edition sc:name ?editionYear.
   ?country sc:name ?winnerCountry .
}
GROUP BY ?country ?winnerCountry
HAVING (COUNT(?country)>=2)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('winnerCountry', 'Italy')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('winnerCountry', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('winnerCountry', 'Brazil')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('winnerCountry', 'Argentina')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('winnerCountry', 'France')]
[('country', 'http://www.wikidata.org/entity/Q77'), ('winnerCountry', 'Uruguay')]
6


In [60]:
# write your queries
# find winner (P1346)
# [('p', 'http://www.wikidata.org/prop/direct/P3450'), ('objname', 'FIFA World Cup')] sports season of league or competition (P3450)
# [('obj', 'http://www.wikidata.org/entity/Q19317'), ('objname', 'FIFA World Cup')]
# let's find who win fifa world cup
# let's find those countries who win world cup 2 or more then 2 times (#winner)
queryString = """
SELECT DISTINCT ?winnerCountry (COUNT(?country) as ?country) 
WHERE {
   # bind something
   ?edition wdt:P3450 wd:Q19317 ;
       wdt:P1346 ?winner.
       

   ?winner wdt:P1532 ?country; #find sport country
       wdt:P17 ?country .
   # get the label
   
   ?country sc:name ?winnerCountry .
   ?edition sc:name ?editionYear.
}
GROUP BY ?country ?winnerCountry
HAVING (COUNT(?country)>=2)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('winnerCountry', 'Italy'), ('country', '4')]
[('winnerCountry', 'Germany'), ('country', '4')]
[('winnerCountry', 'Brazil'), ('country', '5')]
[('winnerCountry', 'Argentina'), ('country', '2')]
[('winnerCountry', 'France'), ('country', '2')]
[('winnerCountry', 'Uruguay'), ('country', '2')]
6


In [61]:
# write your queries
# find winner (P1346)
# [('p', 'http://www.wikidata.org/prop/direct/P3450'), ('objname', 'FIFA World Cup')] sports season of league or competition (P3450)
# [('obj', 'http://www.wikidata.org/entity/Q19317'), ('objname', 'FIFA World Cup')]
# let's find who win fifa world cup
# let's find those countries who win world cup 2 or more then 2 times.
queryString = """
SELECT DISTINCT ?country ?winnerCountry
WHERE {
   # bind something
   ?edition wdt:P3450 wd:Q19317 ;
       wdt:P1346 ?winner.
       

   ?winner wdt:P1532 ?country; #find sport country
       wdt:P17 ?country .
   # get the label
   ?edition sc:name ?editionYear.
   ?country sc:name ?winnerCountry .
}
GROUP BY ?country ?winnerCountry
HAVING (COUNT(?country)>=2)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('winnerCountry', 'Italy')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('winnerCountry', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('winnerCountry', 'Brazil')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('winnerCountry', 'Argentina')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('winnerCountry', 'France')]
[('country', 'http://www.wikidata.org/entity/Q77'), ('winnerCountry', 'Uruguay')]
6


In [62]:
# write your queries
# find winner (P1346)
# [('p', 'http://www.wikidata.org/prop/direct/P3450'), ('objname', 'FIFA World Cup')] sports season of league or competition (P3450)
# [('obj', 'http://www.wikidata.org/entity/Q19317'), ('objname', 'FIFA World Cup')]
# let's find who win fifa world cup
# let's find those countries who win world cup 2 or more then 2 times.
# let's take data from query no.5.
# here we need to make a subquery of query no.5
queryString = """
SELECT DISTINCT ?club ?clubName
WHERE {
    ?s wdt:P3450 wd:Q18756;
        wdt:P1346 ?club .
        
    ?club wdt:P17 ?country .
   # get the label
   ?club sc:name ?clubName .
   {
       SELECT DISTINCT ?country ?winnerCountry
        WHERE {
               # bind something
               ?edition wdt:P3450 wd:Q19317 ;
                   wdt:P1346 ?winner.
       

       ?winner wdt:P1532 ?country; #find sport country
       wdt:P17 ?country .
       # get the label
       #?edition sc:name ?editionYear.
       ?country sc:name ?winnerCountry .
    }
    GROUP BY ?country ?winnerCountry
    HAVING (COUNT(?country)>=2)
   }
}


LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('club', 'http://www.wikidata.org/entity/Q132885'), ('clubName', 'Olympique de Marseille')]
[('club', 'http://www.wikidata.org/entity/Q15789'), ('clubName', 'FC Bayern Munich')]
[('club', 'http://www.wikidata.org/entity/Q41420'), ('clubName', 'Borussia Dortmund')]
[('club', 'http://www.wikidata.org/entity/Q51974'), ('clubName', 'Hamburger SV')]
[('club', 'http://www.wikidata.org/entity/Q1422'), ('clubName', 'Juventus F.C.')]
[('club', 'http://www.wikidata.org/entity/Q1543'), ('clubName', 'A.C. Milan')]
[('club', 'http://www.wikidata.org/entity/Q631'), ('clubName', 'FC Internazionale Milano')]
7


Final query for this task

In [62]:
# # write your final query
# find winner (P1346)
# [('p', 'http://www.wikidata.org/prop/direct/P3450'), ('objname', 'FIFA World Cup')] sports season of league or competition (P3450)
# [('obj', 'http://www.wikidata.org/entity/Q19317'), ('objname', 'FIFA World Cup')]
# let's find who win fifa world cup
# let's find those countries who win world cup 2 or more then 2 times.
# let's take data from query no.5.
# here we need to make a subquery of query no.5
queryString = """
SELECT DISTINCT ?club ?clubName
WHERE {
    ?s wdt:P3450 wd:Q18756;
        wdt:P1346 ?club .
        
    ?club wdt:P17 ?country .
   # get the label
   ?club sc:name ?clubName .
   {
       SELECT DISTINCT ?country ?winnerCountry
        WHERE {
               # bind something
               ?edition wdt:P3450 wd:Q19317 ;
                   wdt:P1346 ?winner.
       

       ?winner wdt:P1532 ?country; #find sport country
       wdt:P17 ?country .
       # get the label
       #?edition sc:name ?editionYear.
       ?country sc:name ?winnerCountry .
    }
    GROUP BY ?country ?winnerCountry
    HAVING (COUNT(?country)>=2)
   }
}


LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('club', 'http://www.wikidata.org/entity/Q132885'), ('clubName', 'Olympique de Marseille')]
[('club', 'http://www.wikidata.org/entity/Q15789'), ('clubName', 'FC Bayern Munich')]
[('club', 'http://www.wikidata.org/entity/Q41420'), ('clubName', 'Borussia Dortmund')]
[('club', 'http://www.wikidata.org/entity/Q51974'), ('clubName', 'Hamburger SV')]
[('club', 'http://www.wikidata.org/entity/Q1422'), ('clubName', 'Juventus F.C.')]
[('club', 'http://www.wikidata.org/entity/Q1543'), ('clubName', 'A.C. Milan')]
[('club', 'http://www.wikidata.org/entity/Q631'), ('clubName', 'FC Internazionale Milano')]
7
